<a href="https://colab.research.google.com/github/Idaogah/datalab/blob/main/image_color_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install --upgrade webcolors
!pip uninstall webcolors -y
!pip install webcolors==1.12

Found existing installation: webcolors 1.12
Uninstalling webcolors-1.12:
  Successfully uninstalled webcolors-1.12
  Using cached webcolors-1.12-py3-none-any.whl.metadata (2.0 kB)
Using cached webcolors-1.12-py3-none-any.whl (9.9 kB)


In [2]:

# Mount Drive
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import webcolors
import numpy as np
from PIL import Image

# Load the image
image_path = '/content/drive/My Drive/ML_AI/'
image = Image.open(image_path + 'text_color.jpg')

# Convert image to RGB
rgb_image = image.convert("RGB")

# Getting Text Color

In [14]:

# Get the pixel values at a region around the text
cropped_region = rgb_image.crop((30, 20, 200, 40))  # Adjusted coordinates based on image structure
cropped_pixels = np.array(cropped_region)

# Get the most common color in the cropped region
average_color = np.mean(cropped_pixels, axis=(0, 1))

# Convert the average color to both hex and RGB values
def rgb_to_hex_and_rgb(rgb):
    hex_value = "#{:02x}{:02x}{:02x}".format(int(rgb[0]), int(rgb[1]), int(rgb[2]))
    rgb_value = (int(rgb[0]), int(rgb[1]), int(rgb[2]))
    return hex_value, rgb_value

average_color_hex, average_color_rgb = rgb_to_hex_and_rgb(average_color)

# Function to get the closest color name
def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

# Try to find the exact or closest color name
def get_color_name(rgb_value):
    try:
        # Try to find the exact color name
        return webcolors.rgb_to_name(rgb_value)
    except ValueError:
        # If there's no exact match, return the closest color name
        return closest_color(rgb_value)

# Get the color name for the extracted RGB value
color_name = get_color_name(average_color_rgb)

# Output both the hex, RGB, and color name
average_color_hex, average_color_rgb, color_name

# Print the average color in hex format
print("The text color in the screenshot has a hex code of " + average_color_hex + \
      ".\n The closest color name is " + color_name + \
      ". \nThe RGB color code is " + str(average_color_rgb))

The text color in the screenshot has a hex code of #d9d9da.
 The closest color name is gainsboro. 
The RGB color code is (217, 217, 218)


# Getting Image Back ground color

In [26]:
# To find the background color, we'll sample a different region of the image that likely contains background pixels.
# Choosing a region slightly away from the text.

background_region = rgb_image.crop((0, 0, 20, 20))  # Coordinates adjusted to capture background
background_pixels = np.array(background_region)

# Get the most common color in the cropped region
average_background_color = np.mean(background_pixels, axis=(0, 1))

# Convert the average background color to a hex code
average_background_color_hex = rgb_to_hex_and_rgb(average_background_color)
color_name = get_color_name(average_background_color)

print("The background color of the screenshot is #" + color_name + \
      ",\n and Hex and RGB codes are" + str(average_background_color_hex))


The background color of the screenshot is #white,
 and Hex and RGB codes are('#ffffff', (255, 255, 255))


# Let's try another image

In [27]:
# Let's try the function on another image

image2 = Image.open(image_path + 'color_g.jpg')    # getting image from drive

hex_color, rgb_color = rgb_to_hex_and_rgb(np.mean(np.array(image2), axis=(0, 1)))     # analyze using the previous function
imgae2_name = get_color_name(rgb_color)

hex_color, rgb_color, imgae2_name

print("Color Name:", imgae2_name)
print("Hex Color", hex_color)
print("RGB Color", rgb_color)

Color Name: lime
Hex Color #00ef00
RGB Color (0, 239, 0)
